In [1]:
from collections import Counter
import math
from operator import itemgetter


def triplets(s: bytes):
    s = b' ' + s.strip() + b' '
    memview = memoryview(s)
    for i in range(len(s)-2):
        yield memview[i:i+3]
        

def all_triplets_from_many_lines(lines):
    for line in lines:
        yield from triplets(line)
        

def load_file(filename):
    with open(filename, 'rb') as file:
        return {line.rstrip(b'\r\n'): i for i, line in enumerate(file.readlines())}
    
    
def account_triplets(lines):
    c = Counter(all_triplets_from_many_lines(lines))
    m = max(c.values())
    for key in c:
        c[key] /= m  # Нормализуем по максимальному значению
    return c


def get_score(s: bytes):
    # return sum(c[t] for t in triplets(s)) / len(s)
    return sum(c[t] for t in triplets(s)) / math.log(len(s)+1)  # Преимущество длинным строкам

In [2]:
old_file = load_file('../stringdumps/stringdump_0_47_04.txt')
old_file.update(load_file('../stringdumps/stringdump_0_47_05.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_47_03.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_47_02.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_47_01.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_44_12.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_01.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_02.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_05.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_06.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_08.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_09.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_10.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_50_11.txt'))

c = account_triplets(old_file)  # Обучаем на старых файлах

In [3]:
new_file = load_file('../stringdumps/stringdump.txt')

In [4]:
diff = sorted(new_file.keys()-old_file.keys(), key=lambda s: get_score(s))

In [5]:
for item in diff:
    print(item, get_score(item))

b'Z0R' 0.0
b'p\xe2\x89\xa4\xc2\xa2' 0.0
b"'t=" 0.0
b'P\xc3\xa8\xc3\x87' 0.0
b'CNp' 0.0
b'\xce\xb1\xc3\xbcI' 0.0
b'p5' 0.0
b'P\xc6\x92\xe2\x95\xac' 0.0
b'\xc3\xa1h"' 0.0
b'%Y-%m-%d-%H-%M-%S' 0.0
b'Pw\xe2\x95\xaa' 0.0
b'h' 0.0
b'4\xc3\xb3V' 0.0
b'4\xc2\xa3V' 0.0
b'46J' 0.0
b'\xe2\x95\xa8R' 0.0
b'P\xc2\xb2x' 0.0
b'l\xe2\x95\x93\xe2\x95\xaa' 0.0
b'\xe2\x94\xb4\xe2\x96\x88A' 0.0
b'\xce\xb1U' 0.0
b"'dH" 0.0
b' u\xc3\xbf' 0.0
b'2PA' 0.0
b'0Y' 0.0
b'\xe2\x94\xa4F' 0.0
b'\xe2\x94\x94F' 0.0
b"'dp" 0.0
b"'d\xc3\xbb" 0.0
b'\xe2\x94\xb4F' 0.0
b"'dN" 0.0
b'\xc3\x89\xc2\xb2A' 0.0
b'&Rz' 0.0
b'z\xe2\x89\xa1\xe2\x95\xa5' 0.0
b'\xe2\x89\xa4x\xe2\x88\x9a' 0.0
b'\xc3\x87v\xc3\xa1' 0.0
b'\xe2\x96\x91y' 0.0
b'\xc2\xacu\xe2\x95\xaa' 0.0
b'SR' 0.0
b'k' 0.0
b'\xe2\x89\xa1U' 0.0
b'a\xe2\x95\x96l' 0.0
b'gm[' 0.0
b'p\xc2\xa21' 0.0
b'dA\xc3\xa2' 0.0
b'M:\xe2\x89\xa1\xe2\x95\xa5' 0.0
b'\xe2\x8c\x90a\xc3\xa2' 0.0
b'5A\xe2\x89\xa1\xe2\x95\xa5' 0.0
b'\xc3\xaa\xe2\x94\xbcy\xe2\x89\xa1\xe2\x95\xa5' 0.0
b'\xe2\x95\x9du\x

In [6]:
threshold = 0.04

In [7]:
with open('../stringdumps/stringdump_steam_50_10.txt', 'wb') as output:
    for line, number in sorted(new_file.items(), key=itemgetter(1)):
        if line in diff and get_score(line) < threshold: # Отсеиваем только добавившиеся строки
            continue
        
        output.write(line)
        output.write(b'\n')